In [1]:
import pickle
from tensorflow.data.experimental import load as load_tf_dataset
import tensorflow as tf

In [2]:
with open('./element_spec.pkl', 'rb') as f:
    element_spec = pickle.load(f)

In [3]:
train_data = load_tf_dataset('train_dataset', element_spec=element_spec)
valid_data = load_tf_dataset('valid_dataset', element_spec=element_spec)

In [4]:
train_data

<_LoadDataset shapes: ({input_data: (128, 512), attention_mask: (128, 512)}, (128, 5)), types: ({input_data: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [5]:
from transformers import TFAutoModel

C:\Users\Anith.Shatamraj\PersonalProjects\UdemyTransformers\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
bert_model = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
bert_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [8]:
input_layer1 = tf.keras.layers.Input(shape=(512,), name='input_data', dtype='int32')
input_layer2 = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

In [9]:
bert_layer = bert_model.bert(input_layer1, attention_mask=input_layer2)[1]
dense1 = tf.keras.layers.Dense(1024, activation='relu')(bert_layer)
output = tf.keras.layers.Dense(5, activation='softmax')(dense1)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [10]:
model = tf.keras.models.Model(inputs=[input_layer1, input_layer2], outputs=output)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_data[0][0]                 
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [12]:
model.layers[2].trainable = False

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_data[0][0]                 
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
metric = tf.keras.metrics.Accuracy()

In [15]:
model.compile(optimizer=optimizer, loss=loss,metrics=[metric])

In [16]:
history = model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
 2/59 [>.............................] - ETA: 4:34:53 - loss: 2.3027 - accuracy: 0.0000e+00

ResourceExhaustedError:  OOM when allocating tensor with shape[128,12,512,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node model/bert/encoder/layer_._0/attention/self/MatMul (defined at C:\Users\Anith.Shatamraj\PersonalProjects\UdemyTransformers\venv\lib\site-packages\transformers\models\bert\modeling_tf_bert.py:283) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_17326]

Errors may have originated from an input operation.
Input Source operations connected to node model/bert/encoder/layer_._0/attention/self/MatMul:
 model/bert/encoder/layer_._0/attention/self/transpose_1 (defined at C:\Users\Anith.Shatamraj\PersonalProjects\UdemyTransformers\venv\lib\site-packages\transformers\models\bert\modeling_tf_bert.py:271)

Function call stack:
train_function
